In [ ]:
import os
import tvm
import nnvm
import topi

import nnvm.symbol as sb
from tvm.contrib import graph_runtime

import numpy as np
import tensorflow as tf

EXAMPLE = 1
target, target_host = 'llvm', 'llvm'
ctx = tvm.cpu(0)

os.environ['XDNN_VERBOSE'] = "1"

In [ ]:
import xdnn


In [ ]:
platform = '1525'

config = {
    'platform': platform,
    'xclbin': "../../../../MLsuite/overlaybins/" + platform + "/overlay_4.xclbin",
    'memory': 9,
    'dsp': 96,
    'netcfg': "work/tvm_compiler.json",
    #'fromtensorflow': False,
    'weights_name': "weights", 
    #datadir': "work/weights_data",
    'quantizecfg': "", # Empty for now to test if quantization is needed in this simple example
    'pngfile': "graph.png",
    'verbose': True
}

xdnn.xdnn_frontend.init(
    platform=config['platform'],
    xclbin=config['xclbin'],
    memory=config['memory'],
    dsp=config['dsp'],
    netcfg=config['netcfg'],
    quantizecfg=config['quantizecfg'],
    weights_name=config['weights_name'],
    input_shape=(1,1,4,4),
    pngfile=config['pngfile'],
    verbose=config['verbose']
)

if EXAMPLE == 1:
    input_shape = (1,1,4,4)
    input_type = 'float32'
    output_shape = (1,1,2,2)
    
    data = np.reshape(np.array([[1,1,0,4],[5,1,0,8],
                                [3,5,1,0],[1,9,3,4]]), input_shape)
    x = sb.Variable("x")
    z = sb.max_pool2d(x, pool_size=[2,2], strides=[2,2])
    graph = nnvm.graph.create(z)
    print(graph.index.nodes)
    
    print("\nNNVM COMPILER BUILD\n")
    
    params = {}
    shape_dict = { 'x': input_shape }
    dtype_dict = { 'x': input_type }
    graph, lib, params = nnvm.compiler.build(
                graph, target, shape_dict, dtype_dict,
                params=params, target_host=target_host)
    
    print(graph.index.nodes)
    print(lib)
    print(params)

    print("\nXDNN SETUP FPGA EXECUTER\n")
    #xdnn.xdnn_frontend.setup_fpga_executer()

    print("\nXDNN CREATE GRAPH RUNTIME\n")
    #m = graph_runtime.create(graph, lib, ctx)
    #m.set_input('x', tvm.nd.array(data.astype(input_type)))

    #m.run()

    #tvm_output = m.get_output(0, tvm.nd.empty(((1, 1, 2, 2)), 'float32'))

    #print(tvm_output)
    
elif EXAMPLE == 2:
    pass

In [ ]:
import os
import xfdnn.rt.xdnn as xdnn_lib

os.environ['XDNN_VERBOSE'] = "1"

ret, handles = xdnn_lib.createHandle("../../../../MLsuite/overlaybins/alveo-u200/overlay_3.xclbin")

if ret:                                                         
    print("ERROR: Unable to create handle to FPGA")
else:
    print("INFO: Successfully created handle to FPGA")